In [1]:
import matplotlib.pyplot as plt

import pickle
import numpy as np

In [2]:
with open('sensitive_analysis_1G_156.pkl','r') as inputs:
    data = pickle.load(inputs)

with open('sensitive_analysis_base_renewal.pkl','r') as inputsbase:
    data_base = pickle.load(inputsbase)

In [3]:
data[1]

([0, 0.001, 0.0, 0.0, 21.235, 0.015, 470.861],
 [3.195730480746087, 90.173123343088406],
 array([ 0.83750608,  0.99590746,  0.85115216,  1.30087807,  1.04443115,
         1.0609384 ,  0.91598331,  1.10271588,  1.06841931,  1.17414509,
         1.13281731,  1.19492335,  1.16537992,  1.21470839,  0.94271651,
         1.02174468,  0.98711478,  1.02818247,  0.99319115,  1.03055977,
         0.99413768,  1.04106646,  1.00266508,  1.03326026,  1.0022196 ,
         1.04146302,  1.0047189 ,  1.05573805,  1.02065009,  1.28923561,
         1.10491688,  0.93844852,  0.9250454 ,  0.96059646,  0.93864905,
         0.96124139,  0.93647136,  0.97657306,  0.94866476,  0.9554677 ,
         0.9312207 ,  0.96902825,  0.95024592,  0.96384885,  0.94465159,
         0.98728405,  0.87974312,  0.95617828,  0.93154356,  0.97072252,
         0.94919196,  0.96180818,  0.93641961,  0.97601815,  0.88142749,
         0.96317825,  0.94183476,  0.98444145,  0.89151623,  0.98996147,
         0.96457301,  1.15613736,  

In [14]:
def get_price_stat_ave(data,decision_times): #period average for each sample
    mitigation = []
    utility = []
    price = []
    a650 = []
    from tree import TreeModel 
    for i in range(len(data)): #load corresponding results
        mitigation.append(data[i][-3])
        utility.append(data[i][-2])
        price.append(data[i][-1])
        #a650.append(data[i][2])
    #print(np.mean(a650))
    
    t = TreeModel(decision_times)
    nodes_index = []
    for i in range(t.num_periods):
        nodes_index.append(t.get_nodes_in_period(i)) #get nodes for slicing
    price_stat_ave = np.zeros((len(data), t.num_periods))
    for i in range(len(data)):
        for j in range(t.num_periods):
            price_stat_ave[i,j] = np.average(data[i][-1][nodes_index[j][0]:nodes_index[j][1]+1])
    return price_stat_ave

def get_90_quantile(price_stat_ave):
    list0 = price_stat_ave[:,0]
    list1 = price_stat_ave[:,1]
    list2 = price_stat_ave[:,2]
    list3 = price_stat_ave[:,3]
    list4 = price_stat_ave[:,4]
    list5 = price_stat_ave[:,5]
    list_ = [list0,list1,list2,list3,list4,list5]
    fivepercent = int(len(list0)*0.05)
    for i in range(len(list_)):
        list_[i]=np.sort(list_[i])[fivepercent:-fivepercent] #take the 90% of the sorted results
    list_min = []
    list_max = []
    for i in list_:
        list_min.append(i[0])
        list_max.append(i[-1])   
    return list_min,list_max #take the lower and upper bounds for the test results

In [15]:
decision_times=[0, 15, 45, 85, 185, 285, 385]
price_stat_ave= get_price_stat_ave(data,decision_times)
price_stat_ave_base =get_price_stat_ave(data_base,decision_times)

list_min,list_max = get_90_quantile(price_stat_ave)
plt.figure()
plt.plot(decision_times[:-1],list_min, 'r--')
plt.plot(decision_times[:-1],list_max,'r--')
plt.plot(decision_times[:-1],price_stat_ave_base[0,:],'b')
plt.title('Sensitivity Analysis for Variable x60', size ='xx-large')
plt.ylabel('Price in $', size = 'x-large')
plt.xlabel('Period Index', size = 'x-large')
plt.savefig('sensitivity_analysis_x60')
#plt.xlim(decision_times)